크롤링을 통해서 기준금리 역사 가져오기

In [66]:
#라이브러리 임포트

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
#웹페이지 가져오기

res = requests.get('https://www.bok.or.kr/portal/singl/baseRate/list.do?dataSeCd=01&menuNo=200643')

#웹페이지 파싱하기
soup = BeautifulSoup(res.content,'html.parser')

#필요한 데이터 추출하기
items = soup.select('#content > div.table.tac > table > tbody >tr')


change_year_list = list()
change_date_list = list()
rp_list = list()

for item in items:
    table_list = item.select('td')
    change_year_list.append(table_list[0].get_text())
    change_date_list.append(table_list[1].get_text())
    rp_list.append(table_list[2].get_text())
    
df = pd.DataFrame({
    "year": change_year_list,
    "change_date": change_date_list,
    "rp": rp_list
}, columns=["year", "change_date", "rp"])

df.tail()

,year,change_date,rp
50,2001,07월 05일,4.75
51,2001,02월 08일,5.00
52,2000,10월 05일,5.25
53,2000,02월 10일,5.00
54,1999,05월 06일,4.75


In [67]:
df['month']=df['change_date'].str[0:2] # 월의 정보만 추출
df['date'] = df['change_date'].str[4:6] # 일의 정보만 추출
df = df.astype({'rp':'float64'}) # rp 컬럼 타입 변경
df['rp_date'] = df['year']+df['month']+df['date'] # 새로운 컬럼 생성
df = df.drop(['change_date', 'year','month','date'], axis=1) # 안쓰는 컬럼 제거
df=df.sort_index(ascending=False) # 날짜가 역순으로 되어 있어서 정렬
df['rp_date'] = pd.to_datetime(df['rp_date'], format='%Y-%m-%d %H:%M:%S', errors='raise') # date 타입으로 변경
df = df.set_index('rp_date') # rp_date를 인덱스로 설정

In [69]:
df.head()

,rp
rp_date,
1999-05-06,4.75
2000-02-10,5.00
2000-10-05,5.25
2001-02-08,5.00
2001-07-05,4.75


In [73]:
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)

df.iplot(kind='scatter') # 시각화를 통해 기준금리 동향 파악

In [74]:
df.iplot(kind='bar')  # 시각화를 통해 기준금리 변경 밀도 파악